## Load all the Python Libraries Required

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from keras.applications.mobilenet import MobileNet
import keras.backend as K
from tensorflow.python.framework import graph_util, graph_io
import coremltools

Using TensorFlow backend.


### Set the learning phase using the backend as False and load the model

In [2]:
K.set_learning_phase(False)

# Load the model with the weights trained on Imagenet 
model = MobileNet(weights='imagenet',include_top=True)

### Convert the model into a coreml model

In [3]:
# Convert the loaded Keras model into a CoreML Model
# Provide image_input_names to specify it is an image model
# also provide the bias values specified by the model
mlmodel = coremltools.converters.keras.convert(model, 
                                               input_names=['image'], 
                                               image_input_names=['image'],
                                               output_names=['classLabelProbs', 'classLabel'],
                                               class_labels='labels.txt', 
                                               blue_bias=-1,
                                               green_bias=-1,
                                               red_bias=-1, 
                                               image_scale=2./255)

Output name length mismatch
0 : input_1, <keras.engine.topology.InputLayer object at 0x10d8a3208>
1 : conv1_pad, <keras.layers.convolutional.ZeroPadding2D object at 0xd2b320cc0>
2 : conv1, <keras.layers.convolutional.Conv2D object at 0xd2b320a90>
3 : conv1_bn, <keras.layers.normalization.BatchNormalization object at 0xd2b320c50>
4 : conv1_relu, <keras.layers.core.Activation object at 0xd2b320eb8>
5 : conv_pad_1, <keras.layers.convolutional.ZeroPadding2D object at 0xd2b348208>
6 : conv_dw_1, <keras.layers.convolutional.DepthwiseConv2D object at 0xd2b348e48>
7 : conv_dw_1_bn, <keras.layers.normalization.BatchNormalization object at 0xd2b36acf8>
8 : conv_dw_1_relu, <keras.layers.core.Activation object at 0xd2b449e48>
9 : conv_pw_1, <keras.layers.convolutional.Conv2D object at 0xd2b4887f0>
10 : conv_pw_1_bn, <keras.layers.normalization.BatchNormalization object at 0xd2b488128>
11 : conv_pw_1_relu, <keras.layers.core.Activation object at 0xd2b4f1048>
12 : conv_pad_2, <keras.layers.convoluti

### Save the model as a .mlmodel file which can be deployed

In [4]:
mlmodel.save('ModelZoo/MobileNet.mlmodel')

### Now extract the model using the TensorFlow session to convert it for Android

In [5]:
S = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(S, 
                                                           S.graph.as_graph_def(),
                                                           ['reshape_2/Reshape'])
graph_io.write_graph(constant_graph, 'ModelZoo/', 'MobileNet.pb', as_text=False)

INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Converted 137 variables to const ops.


INFO:tensorflow:Converted 137 variables to const ops.


'ModelZoo/MobileNet.pb'

### To convert the model for TensorFlow Lite

In [6]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input

In [7]:
input_tensor = Input(shape=(224, 224, 3))
model = MobileNet(weights='imagenet',
                    input_tensor=input_tensor,
                    include_top=True)

In [8]:
keras_file = "MobileNet.h5"
L = tf.contrib.saved_model.save_keras_model(model, keras_file)

INFO:tensorflow:Signatures INCLUDED in export for Classify: None


INFO:tensorflow:Signatures INCLUDED in export for Classify: None


INFO:tensorflow:Signatures INCLUDED in export for Regress: None


INFO:tensorflow:Signatures INCLUDED in export for Regress: None


INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']


...

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: MobileNet.h5/temp-b'1551761065'/saved_model.pb


INFO:tensorflow:SavedModel written to: MobileNet.h5/temp-b'1551761065'/saved_model.pb


In [9]:
converter = tf.contrib.lite.TFLiteConverter.from_saved_model(L)
tflite_model = converter.convert()
open('ModelZoo/'+"MobileNet.tflite", "wb").write(tflite_model)

INFO:tensorflow:Restoring parameters from MobileNet.h5/1551761065/variables/variables


INFO:tensorflow:Restoring parameters from MobileNet.h5/1551761065/variables/variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}


INFO:tensorflow:input tensors info: 


...

INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Froze 137 variables.


INFO:tensorflow:Converted 137 variables to const ops.


INFO:tensorflow:Converted 137 variables to const ops.


16897156

### Now load the model and get the details of the converted model
### These are useful when deploying the models on Android

In [11]:
interpreter = tf.contrib.lite.Interpreter(model_path='ModelZoo/MobileNet.tflite')

interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details, output_details)

[{'name': 'input_1', 'index': 103, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}] [{'name': 'reshape_2/Reshape', 'index': 106, 'shape': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
